In [1]:
# end-user

import asyncio
import httpx
import time
import pandas as pd
import numpy as np

async def send_request(client, url, data):
    start_time = time.time()
    try:
        response = await client.post(url, json=data, timeout=60.0)  # افزایش timeout
        end_time = time.time()
        latency = (end_time - start_time) * 1000  # ms
        
        # بررسی اینکه پاسخ معتبر است
        response_data = response.json()
        if "error" in response_data:
            return {"latency": latency, "status": "error", "error": response_data["error"]}
            
        return {"latency": latency, "status": "success", "response": response_data}
        
    except Exception as e:
        end_time = time.time()
        latency = (end_time - start_time) * 1000
        return {"latency": latency, "status": "error", "error": str(e)}

async def benchmark_system(url, system_name, n_requests=30):  # کاهش تعداد درخواست‌ها برای تست اولیه
    data = {"symbol": "AAPL", "interval": "1h"}
    
    async with httpx.AsyncClient() as client:
        # تست اولیه برای اطمینان از سلامت connection
        try:
            test_response = await client.post(url, json=data, timeout=10.0)
            print(f" {system_name} connection test: OK")
        except Exception as e:
            print(f" {system_name} connection test failed: {e}")
            return None
        
        # اجرای درخواستهای اصلی
        tasks = [send_request(client, url, data) for _ in range(n_requests)]
        results = await asyncio.gather(*tasks)
        
        successful = [r for r in results if r["status"] == "success"]
        failed = [r for r in results if r["status"] == "error"]
        
        if len(successful) > 0:
            latencies = [r["latency"] for r in successful]
            avg_latency = sum(latencies) / len(latencies)
            max_latency = max(latencies)
            min_latency = min(latencies)
            
            # محاسبه throughput بر اساس کل زمان تست
            first_request_time = min([r.get('start_time', 0) for r in successful if 'start_time' in r], default=0)
            last_request_time = max([r.get('end_time', 0) for r in successful if 'end_time' in r], default=0)
            
            if last_request_time > first_request_time:
                total_time = last_request_time - first_request_time
                throughput = len(successful) / total_time
            else:
                throughput = len(successful)  # fallback
            
            print(f" {system_name} Performance (n={n_requests}):")
            print(f"   - Successful: {len(successful)}/{n_requests}")
            print(f"   - Failed: {len(failed)}")
            print(f"   - Avg Latency: {avg_latency:.2f} ms")
            print(f"   - Min Latency: {min_latency:.2f} ms")
            print(f"   - Max Latency: {max_latency:.2f} ms")
            print(f"   - Throughput: {throughput:.2f} req/sec")
            
            if failed:
                print(f"   - Errors: {[f['error'] for f in failed[:3]]}")  # نمایش 3 خطای اول
            
            return {
                "architecture": system_name,
                "avg_latency": avg_latency,
                "min_latency": min_latency,
                "max_latency": max_latency,
                "throughput": throughput,
                "successful": len(successful),
                "failed": len(failed),
                "total": n_requests
            }
        else:
            print(f" No successful requests for {system_name}")
            print(f"   Errors: {[f['error'] for f in failed[:5]]}")  # نمایش 5 خطای اول
            return None

# اجرای تستها
print("=== Starting Performance Tests ===")
print("Waiting 5 seconds for servers to stabilize...")
await asyncio.sleep(5)

print("\n" + "="*50)
print("=== RATS Architecture Test ===")
rats_results = await benchmark_system(
    "http://localhost:8000/request-signal", 
    "RATS", 
    n_requests=30  # تعداد کمتر برای تست اولیه
)

print("\n" + "="*50)
print("=== Monolithic Architecture Test ===")
mono_results = await benchmark_system(
    "http://localhost:8002/request-signal-mono", 
    "Monolithic",
    n_requests=30
)

print("\n" + "="*50)
print("=== Final Results ===")

if rats_results and mono_results:
    # ایجاد DataFrame برای نتایج
    results_df = pd.DataFrame([rats_results, mono_results])
    
    # نمایش نتایج
    print(results_df[['architecture', 'avg_latency', 'min_latency', 'max_latency', 
                     'throughput', 'successful', 'failed', 'total']])
    
    # محاسبه بهبود performance
    latency_improvement = ((mono_results['avg_latency'] - rats_results['avg_latency']) / mono_results['avg_latency']) * 100
    throughput_improvement = ((rats_results['throughput'] - mono_results['throughput']) / mono_results['throughput']) * 100
    
    print(f"\n Performance Comparison:")
    print(f"   - Latency Improvement: {latency_improvement:+.1f}%")
    print(f"   - Throughput Improvement: {throughput_improvement:+.1f}%")
    
    # ذخیره نتایج
    results_df.to_csv("performance_results.csv", index=False)
    print(" Results saved to performance_results.csv")
    
elif rats_results:
    print(" Only RATS test completed successfully")
    print(pd.DataFrame([rats_results]))
elif mono_results:
    print(" Only Monolithic test completed successfully")  
    print(pd.DataFrame([mono_results]))
else:
    print(" Both tests failed. Please check server logs.")

=== Starting Performance Tests ===
Waiting 5 seconds for servers to stabilize...

=== RATS Architecture Test ===
 RATS connection test: OK
 RATS Performance (n=30):
   - Successful: 30/30
   - Failed: 0
   - Avg Latency: 10521.31 ms
   - Min Latency: 10270.04 ms
   - Max Latency: 10738.17 ms
   - Throughput: 30.00 req/sec

=== Monolithic Architecture Test ===
 Monolithic connection test: OK
 Monolithic Performance (n=30):
   - Successful: 30/30
   - Failed: 0
   - Avg Latency: 302.86 ms
   - Min Latency: 11.58 ms
   - Max Latency: 351.22 ms
   - Throughput: 30.00 req/sec

=== Final Results ===
  architecture   avg_latency   min_latency   max_latency  throughput  \
0         RATS  10521.305895  10270.043850  10738.174438          30   
1   Monolithic    302.858170     11.582136    351.217985          30   

   successful  failed  total  
0          30       0     30  
1          30       0     30  

 Performance Comparison:
   - Latency Improvement: -3374.0%
   - Throughput Improvement: